Importing the Necessary Libraries

In [2]:
import re
import numpy as np
import random
import warnings
warnings.filterwarnings('ignore')
from keras.src.layers import LSTM, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

Function For Labeling the Sentences


In [3]:
def initial_label(sentence):
    separated_pattern = re.compile(r'(\s|^)(de|ki)(\s|$)', re.IGNORECASE)
    not_separated_pattern = re.compile(r'\w+(de|ki)\w*', re.IGNORECASE)
    if separated_pattern.search(sentence):
        return 1  # 'de' or 'ki' is a separate word
    elif not_separated_pattern.search(sentence):
        return 0  # 'de' or 'ki' is used as a suffix
    return None  # Inconclusive or not found

Function For Extracting the Sentences That Contains the Suffixes

In [4]:
def extract_and_label_sentences(file_path):
    print("Starting sentence extraction and initial labeling...")
    extracted_sentences = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            if ' de ' in line or ' ki ' in line:
                sentences = re.split(r'[.!?…]', line)
                for sentence in sentences:
                    label = initial_label(sentence)
                    if label is not None:
                        extracted_sentences.append((sentence.strip(), label))
    print(f"Extracted {len(extracted_sentences)} sentences.")
    return extracted_sentences

Function For Merging the Separated Suffixes With the Previous Word

In [5]:
def merge_de_ki(sentence):
    return sentence.replace(" de ", "de ").replace(" ki ", "ki ")

Loading the Dataset

In [6]:
file_path = 'wiki_00'

Extracting and initially labeling sentences from file

In [7]:
print("Extracting and initially labeling sentences from file...")
sentences_with_labels = extract_and_label_sentences(file_path)

Extracting and initially labeling sentences from file...
Starting sentence extraction and initial labeling...
Extracted 597659 sentences.


Processing sentences based on labels



In [8]:
print("Processing sentences based on labels...")
processed_sentences = [merge_de_ki(sentence) if label == 1 else sentence
                       for sentence, label in sentences_with_labels]
print(f"Total processed sentences: {len(processed_sentences)}")

Processing sentences based on labels...
Total processed sentences: 597659


In [9]:
# Tokenizer with limited vocabulary size
VOCAB_SIZE = 20000  # Adjust this to a smaller number if needed
tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(processed_sentences)
sequences = tokenizer.texts_to_sequences(processed_sentences)

In [10]:
# Limit the maximum sequence length
MAX_SEQUENCE_LENGTH = 100  # Adjust this based on the average length of your sentences
padded_sequences = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')


In [11]:
# Prepare labels
labels = [label for _, label in sentences_with_labels]

# Combining sequences and labels for undersampling
combined_data = list(zip(padded_sequences, labels))

# Separating data by class for undersampling
separated_data = [data for data in combined_data if data[1] == 1]
non_separated_data = [data for data in combined_data if data[1] == 0]

In [12]:
# Performing undersampling
undersampled_non_separated = random.sample(non_separated_data, len(separated_data))
balanced_data = separated_data + undersampled_non_separated
random.shuffle(balanced_data)

# Extracting balanced sequences and labels
balanced_sequences = [data[0] for data in balanced_data]
balanced_labels = [data[1] for data in balanced_data]


In [13]:
# Use a smaller subset of data for training
SAMPLE_SIZE = 100000  # Adjust this to your preference
sample_indices = random.sample(range(len(balanced_sequences)), SAMPLE_SIZE)
sample_sequences = [balanced_sequences[i] for i in sample_indices]
sample_labels = [balanced_labels[i] for i in sample_indices]

In [14]:
# Splitting data into training and testing sets
train_sequences_sample, test_sequences_sample, train_labels_sample, test_labels_sample = train_test_split(
    sample_sequences, sample_labels, test_size=0.2, random_state=42)


In [15]:
# Converting labels to numpy array
train_labels_sample = np.array(train_labels_sample)
test_labels_sample = np.array(test_labels_sample)

In [16]:
model = Sequential([
    Embedding(VOCAB_SIZE, 32, input_length=MAX_SEQUENCE_LENGTH),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.5),
    Flatten(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [17]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [18]:
# Model training with reduced epochs
model.fit(np.array(train_sequences_sample), train_labels_sample, epochs=10, validation_split=0.2)

Epoch 1/10


2000/2000 [==============================] - 50s 24ms/step - loss: 0.5360 - accuracy: 0.7110 - val_loss: 0.4684 - val_accuracy: 0.7604
Epoch 2/10
2000/2000 [==============================] - 52s 26ms/step - loss: 0.3994 - accuracy: 0.8091 - val_loss: 0.4908 - val_accuracy: 0.7531
Epoch 3/10
2000/2000 [==============================] - 55s 27ms/step - loss: 0.3051 - accuracy: 0.8590 - val_loss: 0.5628 - val_accuracy: 0.7405
Epoch 4/10
2000/2000 [==============================] - 55s 28ms/step - loss: 0.2056 - accuracy: 0.9086 - val_loss: 0.8700 - val_accuracy: 0.7323
Epoch 5/10
2000/2000 [==============================] - 54s 27ms/step - loss: 0.1307 - accuracy: 0.9440 - val_loss: 1.1369 - val_accuracy: 0.7305
Epoch 6/10
2000/2000 [==============================] - 54s 27ms/step - loss: 0.0871 - accuracy: 0.9647 - val_loss: 1.4884 - val_accuracy: 0.7272
Epoch 7/10
2000/2000 [==============================] - 55s 28ms/step - loss: 0.0571 - accuracy: 0.9781 - val_loss: 1.8331

In [19]:
# Evaluating the model
predicted_probabilities = model.predict(np.array(test_sequences_sample)).flatten()
predicted_labels = [1 if prob > 0.5 else 0 for prob in predicted_probabilities]

625/625 [==============================] - 6s 9ms/step


In [20]:
# Evaluation metrics
accuracy = accuracy_score(test_labels_sample, predicted_labels)
precision = precision_score(test_labels_sample, predicted_labels)
recall = recall_score(test_labels_sample, predicted_labels)
f1 = f1_score(test_labels_sample, predicted_labels)

print(f"Model Evaluation:\nAccuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")

Model Evaluation:
Accuracy: 0.7212, Precision: 0.7196, Recall: 0.7196, F1-Score: 0.7196


Test Cases

In [21]:
test_sentences = [
    "Okuldan eve dönünce derslerinide yapacak.",
    "Bu kitapları okuduktan sonra seninkileride alacağım.",
    "Arkadaşlarıyla parkta oynadı ve yorulduklarında eve de döndüler.",
    "Anneside bu durumdan memnun kalmıştı.",
    "Oyun oynarken zamanın nasıl geçtiğinide anlamadılar.",
    "Öğretmende sorunun cevabını bilmiyordu.",
    "Kedileride yanlarına alarak yola çıktılar.",
    "Sende mi geleceksin?",
    "Bu konudaki görüşlerini merak ediyorum.",
    "Dün gördüğümüz filmdeki karakter çok etkileyiciydi.",
    "Karşıdaki evdeki ışıklar hâlâ yanıyor.",
    "Bu konudaki düşüncelerini değiştirebilir miyim?",
    "Toplantıdaki herkes fikrini açıkça belirtti.",
    "Okuldaki sınavlarımız haftaya başlayacak.",
    "Yoldaki engelleri aşarak hedefine ulaştı.",
    "Dolaptaki elbiseleri değiştirmek istiyorum.",
    "Bu akşamki planlarınızı iptal etmek zorunda kaldık.",
    "Balkondaki çiçekler susuz kalmış.",
    "Kütüphanedeki kitaplar çok eski.",
    "Bahçedeki ağaçların altında piknik yapalım."
]

Results For Test Cases

In [22]:
# Process the test sentences as done with the training data
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

# Predict using the model
test_predictions = model.predict(test_padded).flatten()

# Interpret the predictions
predicted_labels = [1 if prob > 0.5 else 0 for prob in test_predictions]

# Display the results
for sentence, label in zip(test_sentences, predicted_labels):
    print(f"Sentence: {sentence} - {'Separate' if label == 1 else 'Not Separate'}")


1/1 [==============================] - 0s 441ms/step
Sentence: Okuldan eve dönünce derslerinide yapacak. - Not Separate
Sentence: Bu kitapları okuduktan sonra seninkileride alacağım. - Not Separate
Sentence: Arkadaşlarıyla parkta oynadı ve yorulduklarında eve de döndüler. - Separate
Sentence: Anneside bu durumdan memnun kalmıştı. - Separate
Sentence: Oyun oynarken zamanın nasıl geçtiğinide anlamadılar. - Not Separate
Sentence: Öğretmende sorunun cevabını bilmiyordu. - Separate
Sentence: Kedileride yanlarına alarak yola çıktılar. - Separate
Sentence: Sende mi geleceksin? - Separate
Sentence: Bu konudaki görüşlerini merak ediyorum. - Not Separate
Sentence: Dün gördüğümüz filmdeki karakter çok etkileyiciydi. - Separate
Sentence: Karşıdaki evdeki ışıklar hâlâ yanıyor. - Not Separate
Sentence: Bu konudaki düşüncelerini değiştirebilir miyim? - Separate
Sentence: Toplantıdaki herkes fikrini açıkça belirtti. - Not Separate
Sentence: Okuldaki sınavlarımız haftaya başlayacak. - Not Separate
Sent